https://en.wikipedia.org/wiki/Gauss%E2%80%93Newton_algorithm

## toy problem, code up the algo myself

In [61]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as pt
import scipy.linalg as la
from scipy.optimize import least_squares
%matplotlib inline

In [29]:
a = 2
b = 0.5

In [30]:
underlying_function = lambda x: a*x**b

In [31]:
x = np.array([1,2,3,4])
y = [underlying_function(x_i) for x_i in x]

In [32]:
def residual(parameter):
    a, b = parameter
    return y - a * x**b

In [41]:
def jacobian(parameter):
    a, b = parameter
    return np.array([
        -x**b,
        -a*np.log(x)*x**b
        ]).T

In [53]:
p = np.array([5.0,1.0])
    

In [54]:
while True:
    J = jacobian(p)
    r = residual(p)
    step = la.inv(J.T@J)@J.T@r
    p -= step
    
    if np.abs(step).max() < 0.0001:
        break

# toy problem, using scipy.optimize.least_squares

In [67]:
x0 = np.array([2, 2])
res_1 = least_squares(residual, x0)
res_1.x




array([2. , 0.5])

In [68]:
res_1.cost


7.784092131320288e-25

In [69]:
res_1.jac


array([[-1.        ,  0.        ],
       [-1.41421356, -1.96051627],
       [-1.7320508 , -3.80570465],
       [-2.00000001, -5.54517752]])

In [70]:
jacobian([2. , 0.5])

array([[-1.        , -0.        ],
       [-1.41421356, -1.96051629],
       [-1.73205081, -3.8057046 ],
       [-2.        , -5.54517744]])

# Real Problem, simulated, my code

In [115]:
X = np.arange(10**5,10**6,10**5)
V = np.random.uniform(10**8,5*10**8,9)
T = np.random.uniform(0.03,0.5,9)
sos = np.random.uniform(10**10,5*10**10,9)
noise = np.random.uniform(-0.0001,0.0001,9)

In [110]:
gamma = 0.314
alpha = 1
delta = 0.267

In [130]:
permanent_impact = gamma * T * np.sign(X) * (X/(V*T))**alpha *(sos/V)**delta 
# permanent_impact = gamma * T * np.sign(X) * (X/(V*T))**alpha *(sos/V)**delta + noise

In [117]:
permanent_impact

array([0.000104  , 0.00070691, 0.00132496, 0.00074938, 0.00439501,
       0.00133901, 0.00201812, 0.0016612 , 0.0036353 ])

In [121]:
def residual(parameter):
    gamma, alpha, delta = parameter
    prediction = gamma * T * np.sign(X) * (X/(V*T))**alpha *(sos/V)**delta
    residual = permanent_impact - prediction
    return residual

In [122]:
def jacobian(parameter):
    gamma, alpha, delta = parameter
    return np.array([
        -T * np.sign(X) * (X/(V*T))**alpha *(sos/V)**delta,
        -gamma * T * np.sign(X) * (X/(V*T))**alpha *(sos/V)**delta * np.log(X/(V*T)),
        -gamma * T * np.sign(X) * (X/(V*T))**alpha *(sos/V)**delta * np.log(sos/V)
        ]).T

In [131]:
p = [0.5, 0.9, 0.3]

In [132]:
while True:
    J = jacobian(p)
    r = residual(p)
    step = la.inv(J.T@J)@J.T@r
    p -= step
    
    if np.abs(step).max() < 0.00001:
        break

In [133]:
p

array([0.314, 1.   , 0.267])

In [136]:
J

array([[-0.00064393,  0.00147131, -0.00068707],
       [-0.00228801,  0.00450366, -0.00357938],
       [-0.0044744 ,  0.00818996, -0.0059065 ],
       [-0.00253162,  0.00425759, -0.00319988],
       [-0.01395243,  0.01722102, -0.02434344],
       [-0.00403837,  0.00573702, -0.00512781],
       [-0.00670212,  0.00982551, -0.00993121],
       [-0.00526802,  0.00823332, -0.00702594],
       [-0.01161176,  0.01288115, -0.0184132 ]])

# Real Problem, using scipy.optimize.least_squares¶

In [134]:
x0 = np.array([0.5, 0.9, 0.3])
res_1 = least_squares(residual, x0)
res_1.x

array([0.31399879, 1.0000006 , 0.26699982])

In [135]:
res_1.jac

array([[-0.00064393,  0.00147131, -0.00068707],
       [-0.00228801,  0.00450366, -0.00357938],
       [-0.0044744 ,  0.00818996, -0.0059065 ],
       [-0.00253162,  0.00425759, -0.00319988],
       [-0.01395243,  0.01722102, -0.02434345],
       [-0.00403837,  0.00573702, -0.00512781],
       [-0.00670212,  0.00982551, -0.00993121],
       [-0.00526802,  0.00823332, -0.00702594],
       [-0.01161176,  0.01288115, -0.01841321]])

In [151]:
J = res_1.jac
standard_error = np.diagonal((la.inv(J.T@J)@(J.T@np.diag(noise**2)@J)@la.inv(J.T@J)))**0.5

In [152]:
standard_error

array([0.02922804, 0.01251994, 0.01323203])